In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from glob import glob
import os
import json 
import torch
from torch import nn
from torchvision import models
from torch.utils.data import Dataset
# from sklearn.metrics import f1_score
# from sklearn.model_selection import train_test_split

In [11]:
import pandas as pd
import glob

# csv_files = sorted(glob('/raid/workspace/honghee/data_weather/train_x/aws/*/*.csv'))
path = r'/raid/workspace/honghee/data_weather/train_x/aws/2019' 
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame

,Unnamed: 0,지점번호,시각,평균 기온,최고 기온,최저 기온,평균 풍속,합성 풍속,합성 풍향,최대 순간 풍속,최대 순간 풍속 풍향,평균 전운량,합계 강수량,평균 현지기압,평균 해면기압,이슬점온도,10분 평균 시정,운고
0,0,90,2019-12-01 01:00:00,8.3,8.4,8.1,2.8,2.8,276.1,7.1,290.5,4.1,0.0,1021.6,1023.8,-8.8,20000.0,4800.0
1,1,90,2019-12-01 02:00:00,7.9,8.3,7.4,2.8,2.7,280.3,7.1,287.6,5.8,0.0,1021.4,1023.6,-7.4,20000.0,3834.0
2,2,90,2019-12-01 03:00:00,7.5,7.8,7.3,1.6,0.7,330.8,4.5,290.5,6.6,0.0,1021.2,1023.4,-5.6,20000.0,3678.0
3,3,90,2019-12-01 04:00:00,7.6,7.8,7.2,1.3,0.7,314.3,4.7,234.1,6.4,0.0,1020.8,1023.0,-3.2,20000.0,3438.0
4,4,90,2019-12-01 05:00:00,7.5,7.7,7.2,2.1,2.0,287.8,5.3,276.4,3.1,0.0,1020.7,1022.9,-2.8,20000.0,3858.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5606395,460795,989,2019-09-30 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5606396,460796,989,2019-09-30 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5606397,460797,989,2019-09-30 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5606398,460798,989,2019-09-30 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# region_csv = pd.read_csv('/raid/workspace/honghee/data_weather/train_x/aws/region_filled.csv')
# region_csv = region_csv[['지점', '지점주소', '위도', '경도', '노장해발고도(m)']][0:628]
# region_csv['권역'] = region_csv['지점주소'].astype(str).str[0:3]
# region_csv = region_csv.drop('지점주소', axis=1)
# region_csv = region_csv.replace('제주특', '제주도')
# region_csv = region_csv.replace('서귀포', '제주도')
# region_csv.to_csv("/raid/workspace/honghee/data_weather/train_x/aws/region.csv", index=False)

region_csv = pd.read_csv("/raid/workspace/honghee/data_weather/train_x/aws/region.csv")
region_csv

,지점,위도,경도,노장해발고도(m),권역
0,90.0,38.2509,128.5647,17.53,강원도
1,93.0,37.9475,127.7547,95.61,강원도
2,95.0,38.1479,127.3042,155.48,강원도
3,96.0,37.2395,131.8698,99.08,경상북
4,98.0,37.9019,127.0607,115.62,경기도
...,...,...,...,...,...
623,966.0,37.1161,126.3856,38.00,인천광
624,967.0,37.1189,126.6128,70.00,인천광
625,968.0,35.0810,129.0450,78.58,부산광
626,969.0,35.0910,129.1270,37.72,부산광


In [9]:
label_csv = pd.read_csv("/raid/workspace/honghee/data_weather/train_y/weather_news/target.csv")
label_csv

,서울특인천광경기도,부산광울산광경상남,대구광경상북,광주광전라남,전라북,대전광세종특충청남,충청북,강원도,제주도
0,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
1,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
2,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
3,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
4,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
...,...,...,...,...,...,...,...,...,...
52603,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '1', '0', '0', '0', '0', '0', '0', ...","['0', '0', '2', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '2', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
52604,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '1', '0', '0', '0', '0', '0', '0', ...","['0', '0', '2', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '2', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
52605,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '2', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '2', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ..."
52606,"['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '0', '0', '0', '0', '0', '0', '0', ...","['0', '0', '2', 

In [ ]:
csv_files = sorted(glob('/raid/workspace/honghee/data_weather/train_x/aws/*/*.csv'))
# print(csv_files)

temp_csv = pd.read_csv(csv_files[0])[csv_features]
# temp_csv = temp_csv.replace('-',np.nan).dropna()
max_arr, min_arr = temp_csv.max().to_numpy(), temp_csv.min().to_numpy()

# print(temp_csv)
# print(max_arr)
# print(min_arr)

# feature 별 최대값, 최솟값 계산
for csv in tqdm(csv_files[1:69]):
    temp_csv = pd.read_csv(csv)[csv_features]
    temp_csv = temp_csv.replace('-',np.nan).dropna()

In [ ]:
# aws csv 데이터에 위도, 경도, 해발고도 및 지점번호와 매칭되는 해당 권역 등 4가지 col 추가

sample_csv = pd.read_csv('/raid/workspace/honghee/data_weather/train_x/aws/2014/201407.csv')

do_list = []
latitude_list = []
longtitude_list = []
elevation_list = []
for i in range(len(sample_csv)):
    region = sample_csv.loc[i, '지점번호']
    if region_csv.index[region_csv['지점'] == region].tolist() == []:  # 지점번호 정보가 없는 지역 존재
        do_list.append('none')
        latitude_list.append('none')
        longtitude_list.append('none')
        elevation_list.append('none')
    else:
        region_index = region_csv.index[region_csv['지점'] == region].tolist()[0]
        do = region_csv.loc[region_index, '권역']
        latitude = region_csv.loc[region_index, '위도']
        longtitude = region_csv.loc[region_index, '경도']
        elevation = region_csv.loc[region_index, '노장해발고도(m)']

        do_list.append(do)
        latitude_list.append(latitude)
        longtitude_list.append(longtitude)
        elevation_list.append(elevation)
        
sample_csv['위도']=latitude_list
sample_csv['경도']=longtitude_list
sample_csv['해발고도']=elevation_list
sample_csv['권역']=do_list
sample_csv = sample_csv.drop(sample_csv[sample_csv.권역 == 'none'].index)
sample_csv.to_csv("/raid/workspace/honghee/data_weather/train_x/aws/sample.csv", index=False)


In [ ]:
# aws 데이터에서 target csv의 label과 매칭되는 시각, 권역의 row에 해당 label 추가

target_csv = pd.read_csv('/raid/workspace/honghee/data_weather/train_y/weather_news/2014_2020_target.csv')
label_csv["시각"] = target_csv["datetime"]

sample_csv = pd.read_csv("/raid/workspace/honghee/data_weather/train_x/aws/sample.csv")
# sample_csv = sample_csv.drop('Unnamed: 0', axis=1)
sample_csv = sample_csv.replace('제주특', '제주도')
sample_csv = sample_csv.replace('서귀포', '제주도')

label_list = []
label_names = list(label_csv.columns)

for i in range(len(sample_csv)):
    time = sample_csv.loc[i, '시각']  
    region_name = sample_csv.loc[i, '권역']  

    label_x = label_csv.index[label_csv['시각'] == time].tolist()[0]
    label_y = [i for i in range(len(label_names)) if region_name in label_names[i]][0]
    label = label_csv.iloc[label_x, label_y]
    label_list.append(label)

sample_csv['weather_label']=label_list
sample_csv.to_csv("/raid/workspace/honghee/data_weather/train_x/aws/201407.csv", index=False)


In [ ]:
# 같은 지점끼리 모은 후 결측값 linear interpolation 처리 

aws1 = pd.read_csv("/raid/workspace/honghee/data_weather/train_x/aws/201407.csv")
regionlist_csv = pd.read_csv('/raid/workspace/honghee/data_weather/train_x/aws/region_filled.csv')
regionlist = regionlist_csv['지점'][0:628].tolist()
for r in regionlist:
    y = aws1.loc[aws1['지점번호'] == r]
    globals()[f'aws_{int(r)}'] = y # aws_지점번호 인 csv 생성 및 변수명 할당
    globals()[f'aws_{int(r)}'] =  globals()[f'aws_{int(r)}'].interpolate()

aws = pd.DataFrame()
for r in regionlist:
    aws = pd.concat([aws, globals()[f'aws_{int(r)}']])
aws = aws.fillna(0) 
aws